In [5]:
import polars as pl

from src.energy_forecast.config import PROCESSED_DATA_DIR

df = pl.read_csv(PROCESSED_DATA_DIR / "dataset_daily.csv")
df

id,datetime,diff,source
str,str,f64,str
"""0c9ad311-b86f-4371-a695-512ca4…","""2022-10-01T00:00:00.000000""",703.0,"""dh"""
"""0c9ad311-b86f-4371-a695-512ca4…","""2022-10-02T00:00:00.000000""",334.0,"""dh"""
"""0c9ad311-b86f-4371-a695-512ca4…","""2022-10-03T00:00:00.000000""",891.0,"""dh"""
"""0c9ad311-b86f-4371-a695-512ca4…","""2022-10-04T00:00:00.000000""",661.0,"""dh"""
"""0c9ad311-b86f-4371-a695-512ca4…","""2022-10-05T00:00:00.000000""",499.0,"""dh"""
…,…,…,…
"""400338PVG""","""2022-03-31T00:00:00.000000""",292.8,"""legacy"""
"""400338PVG""","""2022-04-01T00:00:00.000000""",288.8,"""legacy"""
"""400338PVG""","""2022-04-02T00:00:00.000000""",247.2,"""legacy"""


In [7]:
df_id = df.filter(pl.col("id") == "3bf6985d-e014-11eb-9d61-02b402f0c1de")

In [30]:
cum_zeros = df_id.with_columns(
    pl.col("diff").cum_count().over(
        pl.when(pl.col("diff") != 0).then(1).cum_sum().forward_fill()) -1
).with_row_index()

In [33]:
cum_zeros.select(["index", "diff"]).iter_rows()

<generator object DataFrame.iter_rows at 0x756475347760>

In [23]:
cum_zeros.filter(pl.col("diff") >= 14)

index,diff
u32,u32
177,14


In [28]:
df_id.with_row_index().filter(~(pl.col("index").is_in(range(177-(14-1), 177+1))))

index,id,datetime,diff,source
u32,str,str,f64,str
0,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-13T00:00:00.000000""",136.73,"""kinergy"""
1,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-14T00:00:00.000000""",337.87,"""kinergy"""
2,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-15T00:00:00.000000""",325.44,"""kinergy"""
3,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-16T00:00:00.000000""",319.79,"""kinergy"""
4,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-17T00:00:00.000000""",376.29,"""kinergy"""
…,…,…,…,…
515,"""3bf6985d-e014-11eb-9d61-02b402…","""2023-01-27T00:00:00.000000""",1332.27,"""kinergy"""
516,"""3bf6985d-e014-11eb-9d61-02b402…","""2023-01-28T00:00:00.000000""",1331.14,"""kinergy"""
517,"""3bf6985d-e014-11eb-9d61-02b402…","""2023-01-29T00:00:00.000000""",1389.9,"""kinergy"""


In [36]:
def filter_flat_lines(df: pl.DataFrame) -> pl.DataFrame:
    df = df.sort(pl.col("datetime"))  # make sure it is in right order
    starting_n = len(df)
    df = df.with_row_index()
    # get a list of number of cumulative zeros and their ending index
    cum_zeros = df.with_columns(
        pl.col("diff").cum_count().over(
            pl.when(pl.col("diff") != 0).then(1).cum_sum().forward_fill()) - 1
    ).filter(pl.col("diff") > 14  # filter everything longer than 2 weeks
                              ).select(["index", "diff"])

    for row in cum_zeros.iter_rows():
        start_idx = row[0] - (row[1] - 1)
        end_idx = row[0] + 1
        df = df.filter(~(pl.col("index").is_in(range(start_idx, end_idx))))

    df.drop(["index"])
    return df

filter_flat_lines(df_id)

index,id,datetime,diff,source
u32,str,str,f64,str
0,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-13T00:00:00.000000""",136.73,"""kinergy"""
1,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-14T00:00:00.000000""",337.87,"""kinergy"""
2,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-15T00:00:00.000000""",325.44,"""kinergy"""
3,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-16T00:00:00.000000""",319.79,"""kinergy"""
4,"""3bf6985d-e014-11eb-9d61-02b402…","""2021-08-17T00:00:00.000000""",376.29,"""kinergy"""
…,…,…,…,…
515,"""3bf6985d-e014-11eb-9d61-02b402…","""2023-01-27T00:00:00.000000""",1332.27,"""kinergy"""
516,"""3bf6985d-e014-11eb-9d61-02b402…","""2023-01-28T00:00:00.000000""",1331.14,"""kinergy"""
517,"""3bf6985d-e014-11eb-9d61-02b402…","""2023-01-29T00:00:00.000000""",1389.9,"""kinergy"""
